# Training Data Debug

In [15]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy import spatial
from matplotlib import cm

import torch
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from tqdm.auto import tqdm

from sphere import Sphere
from visualize import Visualize
from dh_grid import DHGrid
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the previous (old) data format

In [5]:
export_ds = '/media/scratch/berlukas/nuscenes_old'

cloud_filename = f"{export_ds}/clouds1.npy"
sem_clouds_filename = f"{export_ds}/new_sem_classes_gt1.npy"

print(f"Loading clouds from {cloud_filename}")
cloud_features_old = np.load(cloud_filename)
sem_cloud_features_old = np.load(sem_clouds_filename)
print(f"Shape of clouds is {cloud_features.shape} and sem clouds is {sem_cloud_features.shape}")

Loading clouds from /media/scratch/berlukas/nuscenes_old/clouds1.npy
Shape of clouds is (4766, 2, 400, 400) and sem clouds is (4766, 400, 400)


# Load the current data format

In [3]:
export_ds = '/media/scratch/berlukas/nuscenes'

cloud_filename = f"{export_ds}/clouds.npy"

print(f"Loading clouds from {cloud_filename}")
cloud_features = np.load(cloud_filename)
sem_cloud_features = cloud_features[:, 2, :, :]
cloud_features = cloud_features[:, 0:2, :, :]
print(f"Shape of clouds is {cloud_features.shape}")

Loading clouds from /media/scratch/berlukas/nuscenes/clouds.npy
Shape of clouds is (4766, 2, 400, 400)


## Compare the two datasets

In [13]:
def mapIntensityToRGB(i):
    mask = np.where(i < 0)    
    colors = cm.jet(plt.Normalize(min(i), max(i))(i))
    colors[mask] = 0
    return colors

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)        
    o3d.visualization.draw_geometries([pcd])
    
def visualize_sphere(cloud, ndim = 1):    
    bw = 100
    _, points = DHGrid.CreateGrid(bw)
    cur_sem_cloud = np.reshape(cloud, (ndim, -1)).T
    points_xyz = DHGrid.ConvertPointsToEuclidean(points)        
    points_xyzi = np.column_stack((points_xyz, cur_sem_cloud[:,0]))
    visualize_pointcloud(points_xyzi)
    

In [16]:
visualize_sphere(sem_cloud_features_old[0,:,:], 1)

RuntimeError: [1;31m[Open3D ERROR] GLFW Error: X11: The DISPLAY environment variable is missing[0;m